# Least Squares Monte Carlo for pricing American-style options

Here, we implement the Least Squares Monte Carlo simulation method described by Longstaff-Schwartz (2001). We will assume geometric brownian motion and volatility forecasts from the GARCH model. Afterwards, we will use the simulation to price derivatives of SPX and compare to market prices at the time. The approach used is a dynamic programming approach that essentially breaks down the problem into deciding whether to hold or to excersize at each step, here this is done by fitting the regression model to estimate future payoff from continuation. 

In [225]:
import numpy as np
import pandas as pd
from scipy.special import comb
from sklearn.linear_model import LinearRegression

In [226]:
s = 1
n = 8
r = 0.06
sigma = 0.2
T = 5
r_daily = r / 365
sigma_daily = sigma / 365**0.5
type = 'put'

In [227]:
def initiate_path(npaths = 8, s = 100, r = 0.06, sigma = 0.2, T = 5, strike = 100, type='put'):

    path = pd.DataFrame(np.zeros(npaths) + s)
    r_daily = r/365
    sigma_daily = sigma/365**0.5

    for i in range(T):
        day = path.shape[1] - 1
        current = path.iloc[:,day]
        next = current * np.exp(r_daily - sigma_daily**2/2 + sigma_daily * np.random.normal(size = npaths))
        path = pd.concat([path, next], axis=1)

    path.columns = list(range(T+1))

    if type == 'put':
        path['gain_at_maturity'] = strike - path[T]
    else:
        path['gain_at_maturity'] = path[T] - strike
    
    path['gain_at_maturity'] = path['gain_at_maturity'].apply(lambda x:max(x, 0))
    path['value_at_maturity'] = path['gain_at_maturity']
    path['executed'] = T
    return path

In [228]:
path = initiate_path()
path

,0,1,2,3,4,5,gain_at_maturity,value_at_maturity,executed
0,100.0,100.094766,99.651923,100.052601,100.266441,102.135128,0.000000,0.000000,5
1,100.0,98.858424,97.255987,96.964178,96.437704,96.288020,3.711980,3.711980,5
2,100.0,99.584591,99.269197,99.434107,101.044358,102.553995,0.000000,0.000000,5
3,100.0,100.480087,103.111417,104.284638,105.401832,104.935330,0.000000,0.000000,5
4,100.0,100.030696,99.544457,98.889492,99.506772,99.753833,0.246167,0.246167,5
5,100.0,101.551876,102.180018,102.767887,102.031048,102.779439,0.000000,0.000000,5
6,100.0,100.554893,102.067171,100.755410,99.493924,99.270123,0.729877,0.729877,5
7,100.0,101.364199,101.222010,100.797510,100.796091,102.260489,0.000000,0.000000,5


In this table, each row represents one of the simulated paths of the underlying asset's price over the specified period (5 days). Columns 0 to 5 represent the simulated asset prices over 5 days, for each row, it starts with the initial asset price and changes over time based on the specified drift and volatility, assuming geometric brownian motion. 
The gain_at_maturity column shows the payoff of the option as the difference between the asset and strike prices, but only if it is below the strike price.
Here, they are all executed on the last day because we have not yet implement the early execution decision logic. 

In [229]:
reg = LinearRegression()

# for generation of polynomial series

def polynomial(n, data):
    result = data
    if n == 1:
        return result.values.reshape(-1, 1)
    for i in range(2, n+1):
        result = pd.concat([result, data**i], axis = 1)
    return result


In [230]:
def continuation_values(i, path = path, basis = polynomial, n=1, display=True, strike=100, option_type='put', r=0.06):
    r_daily = r / 365
    if 'executed_on' not in path.columns:
        path['executed_on'] = 'not executed'
    
    day = path[[i, 'value_at_maturity']].copy()
    if option_type == 'put':
        day = day[day[i] < strike]
    else:
        day = day[day[i] > strike]
    
    if day.shape[0] == 0:
        return
    
    transform = basis(n=n, data=day[i])
    reg = LinearRegression()
    reg.fit(transform, day['value_at_maturity'])
    
    if display:
        print('\nDay', i, 'ITM path:', list(day.index))
        print('Conditional expectation function:', 'intercept:', reg.intercept_, 'coef:', reg.coef_)
    
    day['expected_continuation_value'] = reg.predict(transform) * (1 / (1 + r_daily))
    
    if option_type == 'put':
        day['execute_value'] = day[i].apply(lambda x: max(0, strike - x))
    else:
        day['execute_value'] = day[i].apply(lambda x: max(0, x - strike))
    
    day['executed'] = day['execute_value'] > day['expected_continuation_value']
    
    if display:
        print('\nChoose to execute on these paths:', day[day['executed']==True].index.tolist())
    
    def update(row):
        if row['executed']:
            return row['execute_value'] * (1 + r_daily) ** (T - i)
        else:
            return row['value_at_maturity']
    
    day['new_value_at_maturity'] = day.apply(update, axis=1)
    path.loc[day.index, 'value_at_maturity'] = day['new_value_at_maturity']
    path.loc[day[day['executed'] == True].index, 'executed_on'] = i


In [231]:
for day in range(T-1, 0, -1):
    continuation_values(day)
    display(path)


Day 4 ITM path: [1, 4, 6]
Conditional expectation function: intercept: 105.27674580243074 coef: [-1.05315427]

Choose to execute on these paths: [4, 6]


,0,1,2,3,4,5,gain_at_maturity,value_at_maturity,executed,executed_on
0,100.0,100.094766,99.651923,100.052601,100.266441,102.135128,0.000000,0.000000,5,not executed
1,100.0,98.858424,97.255987,96.964178,96.437704,96.288020,3.711980,3.711980,5,not executed
2,100.0,99.584591,99.269197,99.434107,101.044358,102.553995,0.000000,0.000000,5,not executed
3,100.0,100.480087,103.111417,104.284638,105.401832,104.935330,0.000000,0.000000,5,not executed
4,100.0,100.030696,99.544457,98.889492,99.506772,99.753833,0.246167,0.493309,5,4
5,100.0,101.551876,102.180018,102.767887,102.031048,102.779439,0.000000,0.000000,5,not executed
6,100.0,100.554893,102.067171,100.755410,99.493924,99.270123,0.729877,0.506159,5,4
7,100.0,101.364199,101.222010,100.797510,100.796091,102.260489,0.000000,0.000000,5,not executed



Day 3 ITM path: [1, 2, 4]
Conditional expectation function: intercept: 153.70172735099354 coef: [-1.54730378]

Choose to execute on these paths: [2, 4]


,0,1,2,3,4,5,gain_at_maturity,value_at_maturity,executed,executed_on
0,100.0,100.094766,99.651923,100.052601,100.266441,102.135128,0.000000,0.000000,5,not executed
1,100.0,98.858424,97.255987,96.964178,96.437704,96.288020,3.711980,3.711980,5,not executed
2,100.0,99.584591,99.269197,99.434107,101.044358,102.553995,0.000000,0.566079,5,3
3,100.0,100.480087,103.111417,104.284638,105.401832,104.935330,0.000000,0.000000,5,not executed
4,100.0,100.030696,99.544457,98.889492,99.506772,99.753833,0.246167,1.110873,5,3
5,100.0,101.551876,102.180018,102.767887,102.031048,102.779439,0.000000,0.000000,5,not executed
6,100.0,100.554893,102.067171,100.755410,99.493924,99.270123,0.729877,0.506159,5,4
7,100.0,101.364199,101.222010,100.797510,100.796091,102.260489,0.000000,0.000000,5,not executed



Day 2 ITM path: [0, 1, 2, 4]
Conditional expectation function: intercept: 139.8204537840179 coef: [-1.39970356]

Choose to execute on these paths: [0]


,0,1,2,3,4,5,gain_at_maturity,value_at_maturity,executed,executed_on
0,100.0,100.094766,99.651923,100.052601,100.266441,102.135128,0.000000,0.348249,5,2
1,100.0,98.858424,97.255987,96.964178,96.437704,96.288020,3.711980,3.711980,5,not executed
2,100.0,99.584591,99.269197,99.434107,101.044358,102.553995,0.000000,0.566079,5,3
3,100.0,100.480087,103.111417,104.284638,105.401832,104.935330,0.000000,0.000000,5,not executed
4,100.0,100.030696,99.544457,98.889492,99.506772,99.753833,0.246167,1.110873,5,3
5,100.0,101.551876,102.180018,102.767887,102.031048,102.779439,0.000000,0.000000,5,not executed
6,100.0,100.554893,102.067171,100.755410,99.493924,99.270123,0.729877,0.506159,5,4
7,100.0,101.364199,101.222010,100.797510,100.796091,102.260489,0.000000,0.000000,5,not executed



Day 1 ITM path: [1, 2]
Conditional expectation function: intercept: 431.98671286763204 coef: [-4.33220271]

Choose to execute on these paths: []


,0,1,2,3,4,5,gain_at_maturity,value_at_maturity,executed,executed_on
0,100.0,100.094766,99.651923,100.052601,100.266441,102.135128,0.000000,0.348249,5,2
1,100.0,98.858424,97.255987,96.964178,96.437704,96.288020,3.711980,3.711980,5,not executed
2,100.0,99.584591,99.269197,99.434107,101.044358,102.553995,0.000000,0.566079,5,3
3,100.0,100.480087,103.111417,104.284638,105.401832,104.935330,0.000000,0.000000,5,not executed
4,100.0,100.030696,99.544457,98.889492,99.506772,99.753833,0.246167,1.110873,5,3
5,100.0,101.551876,102.180018,102.767887,102.031048,102.779439,0.000000,0.000000,5,not executed
6,100.0,100.554893,102.067171,100.755410,99.493924,99.270123,0.729877,0.506159,5,4
7,100.0,101.364199,101.222010,100.797510,100.796091,102.260489,0.000000,0.000000,5,not executed


In [232]:
# option value on day 0
print('Simulated price under default setting:')
path['value_at_maturity'].mean() * (1+r_daily)**(-T)

Simulated price under default setting:


0.7797762481681497

In [233]:
def option_price(basis = polynomial, n = 1, display = False, npaths = 10000, s = 100, r = 0.06, sigma =0.2, T = 5, strike = 100, type = 'put'):
    path = initiate_path(npaths=npaths, s=s, r=r, sigma=sigma, T=T, strike=strike, type=type)

    r_daily = r / 365
    #T = path.shape[1] - 4

    for day in range(T-1, 0, -1):
        continuation_values(i=day, path=path, basis=basis, n=n, strike=strike, option_type=type, display=False)

    if display:
        display(path)

    discounted_values = path['value_at_maturity'] * (1 + r_daily) ** (-T)
    mean_val = np.mean(discounted_values)

    return mean_val

## Pricing Compared to Market Data

The S&P 500 closed at 3,852.97 on January 4, 2023. Looking at the SPX option prices at the time, we see that the put contract expiring on February 2, 2023, for a strike price of 3750, had a bid of 49.1 and an ask of 49.8 (data from optionsDX). 
The US 1 Month Treasury bond yield was 4.20%. 
Our forecasted volatility was around 0.22 at that time.


In [234]:
mean_value = option_price(basis = polynomial, n = 2, strike = 3750, type='put', s = 3852.97, T = 29, r = 0.042, sigma = 0.22, npaths = 10000)
print(f"Mean Option Price using 2 term polynomial basis with 10,000 paths: ${mean_value:.2f}")

Mean Option Price using 2 term polynomial basis with 10,000 paths: $48.21
